In [6]:
import pandas as pd
from utils import get_control_clean
from isc import isc_b
import math
import pickle

In [7]:
def get_intertisial_data(clean_t, clean_c, target_var, out_suffix):
    treated = pd.read_csv(clean_t, index_col=0)
    controls = pd.read_csv(clean_c, index_col=0)
    print(len(treated.pidp.unique()))
    target_var = target_var
    samples = get_control_clean(controls, treated,
                                [target_var,
                                'dvage',
                                'mastat_recoded',
                                'asian',
                                'black',
                                'mixed',
                                'other',
                                'low',
                                'middle'],
                                target_var,
                                'weight_yearx')
    #with open(f'./outputs/{target_var}_{out_suffix}.pkl', 'wb') as file:
    #    pickle.dump(samples, file)
    return samples

In [13]:
def run_isc(clean_t, clean_c, target_var, out_suffix, k_n=35):
    samples = get_intertisial_data(clean_t, clean_c, target_var, out_suffix)
    out = isc_b(samples, penalized=False, reduction=True, k_n=k_n, placebo=True)
    print(out['synths'])
    #diffs = pd.concat(out['diffs'], axis=1).sort_index()
    #w_diffs = pd.concat(out['weighted_diff'], axis=1).sort_index()
    #treats = pd.concat(out['treats'], axis=1).sort_index()
    #synths = pd.concat(out['synths'], axis=1).sort_index()
    #diffs.to_csv(f'./outputs/diffs_{target_var}_{out_suffix}.csv')
    #w_diffs.to_csv(f'./outputs/w_diffs_{target_var}_{out_suffix}.csv')
    #treats.to_csv(f'./outputs/treats_{target_var}_{out_suffix}.csv')
    #synths.to_csv(f'./outputs/synths_{target_var}_{out_suffix}.csv')

In [14]:
def run_isc_short(samples_pkl, target_var, out_suffix, k_n=35):
    with open(samples_pkl, 'rb') as file:
        samples = pickle.load(file)
    out = isc_b(samples, penalized=False, custom_v='auto', reduction=True, k_n=k_n)
    diffs = pd.concat(out['diffs'], axis=1).sort_index()
    w_diffs = pd.concat(out['weighted_diff'], axis=1).sort_index()
    treats = pd.concat(out['treats'], axis=1).sort_index()
    synths = pd.concat(out['synths'], axis=1).sort_index()
    #diffs.to_csv(f'./outputs/diffs_{target_var}_{out_suffix}.csv')
    #w_diffs.to_csv(f'./outputs/w_diffs_{target_var}_{out_suffix}.csv')
    #treats.to_csv(f'./outputs/treats_{target_var}_{out_suffix}.csv')
    #synths.to_csv(f'./outputs/synths_{target_var}_{out_suffix}.csv')

In [15]:
run_isc('../data/sub_strata/ii_t_hi_edh.csv', '../data/sub_strata/ii_c_full_edh.csv', 'ind_inc_deflated', 'hi')

87


100%|██████████| 78/78 [00:01<00:00, 51.18it/s]


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
import os

directory_path = "./sub_strata/"
files = os.listdir(directory_path)
for file in files:
    if "_t_" in file:
        shards = file.split("_")
        file_name_c = f'{shards[0]}_c_full_{shards[3]}'
        run_isc(f'{directory_path}{file}', f'{directory_path}{file_name_c}', f'{shards[0]}', f'{shards[2]}_{shards[3]}')